<a href="https://colab.research.google.com/github/wacif/PIAIC_Applied_GenAI_Batch62/blob/main/Quarter_02/_Projects/project_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 02: LangChain RAG Project**


---



---


####  **Task:**

Create a Google Colab Notebook that integrates a function/tool-calling workflow using LangChain, Google Gemini Flash, and custom tools. Your system should:

- **Set Up Environment Variables:** Load the Google Gemini Flash API key securely.
- **Define the Custom Tool:** Implement a set of tools to perform some custom actions.
- **Create the Tool Wrapper for LangChain:** Integrate the calculator with LangChain as a callable tool.
- **Set Up the Google Gemini Flash Model:** Initialize the Gemini model with tool-calling capabilities.
- **Build the Conversational Chain:** Develop a conversational interface that maintains context and interacts with the tools.
- **Test the Custom Tools:** Demonstrate the system's ability to handle various queries.
- **Optional Enhancements:** Extend functionality with advanced operations, improved error handling, UI integration, or logging.

In [1]:
%pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.0 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [21]:
from langchain.tools import tool

@tool
def calculator(expression: str) -> str:
    """
    A simple calculator tool that evaluates mathematical expressions.
    Args:
        expression (str): A mathematical expression to evaluate.
    Returns:
        str: The result of the evaluation.
    """
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {e}"

@tool
def acceleration(velocity: int, time: int) -> float:
    """Find acceleration"""
    return velocity / time

In [22]:
tools = [add, multiply]

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp"
)

# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

In [26]:
# Invoke the llm
message = 'find acceleration where velocity is 5 and time is 50'
response = llm_with_tools.invoke(message)

In [27]:
print(response)

content='' additional_kwargs={'function_call': {'name': 'Acceleration', 'arguments': '{"__arg1": "velocity is 5 and time is 50"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-16847170-5a86-4d0c-99f1-2b1b2b516ae6-0' tool_calls=[{'name': 'Acceleration', 'args': {'__arg1': 'velocity is 5 and time is 50'}, 'id': 'b6fb7256-008c-4ef8-a8d1-881f1ecf38ab', 'type': 'tool_call'}] usage_metadata={'input_tokens': 92, 'output_tokens': 14, 'total_tokens': 106, 'input_token_details': {'cache_read': 0}}
